In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
os.chdir("/content/drive/My Drive")

# **Preprocess**
1.   Picture format conversion (any format to png)

In [ ]:
import os
from PIL import Image

def bmpToPng(file_path,new_file_path):
    for fileName in os.listdir(file_path):
        # print(fileName)
        newFileName = fileName[0:fileName.find(".")]+".png"
        im = Image.open(file_path+"/"+fileName)
        im.save(new_file_path+"/"+newFileName)

file_path = "./data/refuge2/test/disc_small/mask_copy"
new_file_path = "./data/refuge2/test/disc_small/mask"
bmpToPng(file_path,new_file_path)


2.   Resize (512*512)



In [ ]:
from PIL import Image
import os.path
import glob
def convert(file,outdir,width=512,height=512):
    img=Image.open(file)
    try:
        new_img=img.resize((width,height),Image.BILINEAR)   
        new_img.save(os.path.join(outdir,os.path.basename(file)))
    except Exception as e:
        print(e)
for file in glob.glob("./data/test/images_ROI/*.png"):
    print(file)
    convert(file,"./data/test/images/")

./data/test/images_ROI/ROI.png


# **Install segmentation models**

In [ ]:
!pip install segmentation-models

# **Train** 

*   Train set: REFUGE train set + validation set
*   Validation set: REFUGE test set
*   Weight path: ./weight_1.hdf5
*   Backbone: resnet34
*   Optimizer = Adam(lr = 1e-4)
*   Loss: binary_crossentropy
*   Epochs: 100
*   Batch_size = 10




In [ ]:
from segmentation_models import Unet
from segmentation_models import get_preprocessing
from segmentation_models.losses import bce_jaccard_loss
from segmentation_models.metrics import iou_score
import os
import cv2
import segmentation_models as sm
import tensorflow as tf
import keras
import numpy as np
from keras.optimizers import adam
MAX_EPOCHS = 100

BACKBONE = 'resnet34'
preprocess_input = sm.get_preprocessing(BACKBONE)

class Data_Gen(keras.utils.Sequence):

  def __init__(self, batch_size, x_set, y_set):
      self.x, self.y = np.array(x_set), np.array(y_set)
      self.batch_size = batch_size
      #self.indices=np.arange(len(x_set))
      self.indices = np.arange(self.x.shape[0])
      np.random.shuffle(self.indices)
      #self.type = type
      #self.preproc = preproc

  def __len__(self):
      # print(self.type + ' - len : ' + str(int(np.ceil(self.x.shape[0] / self.batch_size))))
      return int(np.ceil(self.x.shape[0] / self.batch_size))

  def __getitem__(self, idx):
      #inds = self.indices[idx * self.batch_size:(idx + 1) * self.batch_size]
      upper_bound = len(self.indices) if ((idx + 1) * self.batch_size) > len(self.indices)  else (idx + 1) * self.batch_size    
      inds = self.indices[idx * self.batch_size:upper_bound]
      batch_x = np.empty((len(inds),512,512,3))    
      batch_y = np.empty((len(inds),512,512,1))
      for i in range(len(inds)):
        image_x = cv2.imread(self.x[inds[i]])
        cv2.normalize(image_x,image_x,alpha = 0,beta = 1,norm_type = cv2.NORM_MINMAX,dtype = cv2.CV_32F)
        batch_x[i] = cv2.cvtColor(image_x,cv2.COLOR_BGR2RGB)  
        if mask_y is not None:
          mask_y = cv2.imread(self.y[inds[i]],cv2.IMREAD_GRAYSCALE)/255 
        else:
          print(self.y[inds[i]])
        mask_y = np.expand_dims(mask_y, axis=2)
        #cv2.normalize(mask_y,mask_y,alpha = 0,beta = 1,norm_type = cv2.NORM_MINMAX,dtype = cv2.CV_32F)
        # y = cv2.imread(self.y[inds[i]],cv2.IMREAD_GRAYSCALE)
        # y = np.reshape(y, (-1,1))
        batch_y[i] = mask_y
      return batch_x, batch_y

  def on_epoch_end(self):
      np.random.shuffle(self.indices)


images_dir = os.path.join('./data/train/', 'image')
masks_dir = os.path.join('./data/train/', 'label')
ids = os.listdir(images_dir)
images_fps  = [os.path.join(images_dir, image_id) for image_id in ids]
print(images_fps)
masks_fps = [os.path.join(masks_dir, mask_id) for mask_id in ids]
print(masks_fps)

test_images_dir = os.path.join('./data/val/', 'image')
test_masks_dir = os.path.join('./data/val/', 'mask')
ids = os.listdir(test_images_dir)
test_images_fps  = [os.path.join(test_images_dir, image_id) for image_id in ids]
print(test_images_fps)
test_masks_fps = [os.path.join(test_masks_dir, mask_id) for mask_id in ids]
print(test_masks_fps)

opt = adam(lr = 1e-4)
model = Unet(BACKBONE, encoder_weights='imagenet')
#model.compile('Adam','mse')
model.compile(optimizer = opt, loss = 'binary_crossentropy', metrics = ['accuracy'])
#model.compile(
#    'Adam',
#    loss=sm.losses.bce_jaccard_loss,
#    metrics=[sm.metrics.iou_score],
#)

batch_size = 10
#preproc = preprocess_input

train_gen = Data_Gen(batch_size, images_fps, masks_fps)
dev_gen = Data_Gen(batch_size, test_images_fps,test_masks_fps)

model_checkpoint = keras.callbacks.ModelCheckpoint('weight_1.hdf5', monitor="val_loss", mode="min", verbose=1, save_best_only=True)
model.fit_generator(train_gen,epochs=MAX_EPOCHS,validation_data=dev_gen,callbacks=[model_checkpoint])

Using TensorFlow backend.


Segmentation Models: using `keras` framework.
['./data/train/image/g0001.png', './data/train/image/g0004.png', './data/train/image/g0003.png', './data/train/image/g0002.png', './data/train/image/g0005.png', './data/train/image/g0006.png', './data/train/image/g0007.png', './data/train/image/g0008.png', './data/train/image/g0012.png', './data/train/image/g0010.png', './data/train/image/g0009.png', './data/train/image/g0011.png', './data/train/image/g0014.png', './data/train/image/g0015.png', './data/train/image/g0013.png', './data/train/image/g0019.png', './data/train/image/g0016.png', './data/train/image/g0017.png', './data/train/image/g0018.png', './data/train/image/g0022.png', './data/train/image/g0021.png', './data/train/image/g0020.png', './data/train/image/g0023.png', './data/train/image/g0024.png', './data/train/image/g0026.png', './data/train/image/g0025.png', './data/train/image/g0030.png', './data/train/image/g0029.png', './data/train/image/g0027.png', './data/train/image/g0028

# **Predict**
*   Weight path: ./weight_1.hdf5
*   Backbone: resnet34

In [ ]:
import skimage.io as io
from segmentation_models import Unet
from segmentation_models import get_preprocessing
from segmentation_models.losses import bce_jaccard_loss
from segmentation_models.metrics import iou_score
import os
import cv2
import segmentation_models as sm
import tensorflow as tf
import keras
import numpy as np
from keras.optimizers import adam

BACKBONE = 'resnet34'
preprocess_input = sm.get_preprocessing(BACKBONE)


def testGenerator(name,test_path):
        img = np.empty((1,512,512,3))
        path = os.path.join(test_path,"%s.png"%name)
        image =cv2.imread(path)
        cv2.normalize(image,image,alpha = 0,beta = 1,norm_type = cv2.NORM_MINMAX,dtype = cv2.CV_32F)
        img[0] = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
        yield img

def labelVisualize(num_class,color_dict,img):
    img = img[:,:,0] if len(img.shape) == 3 else img
    img_out = np.zeros(img.shape + (3,))
    for i in range(num_class):
        img_out[img == i,:] = color_dict[i]
    return img_out / 255

def saveResult(name,save_path,npyfile,flag_multi_class = False,num_class = 2):
    for i,item in enumerate(npyfile):
        img = labelVisualize(num_class,COLOR_DICT,item) if flag_multi_class else item[:,:,0]
        io.imsave(os.path.join(save_path,"%s_predict.png"%name),img)

Using TensorFlow backend.


Segmentation Models: using `keras` framework.


**Specify picture(one picture)**
*   Input: picture name, folder path
*   Output path: ./data/result

In [ ]:
name = "T0001"
file_path = "./data/val/image"
testGene = testGenerator(name,file_path)

model =  Unet(BACKBONE, encoder_weights='imagenet')
model.load_weights("weight_1.hdf5")

results = model.predict_generator(testGene,1,verbose=1)

saveResult(name,"./data/result",results)

1/1 [==============================] - 5s 5s/step


**Multi pictures**

*   Input: folder path
*   Output path: ./data/result



In [ ]:
file_path = "./data/train/image7"
for fileName in os.listdir(file_path):
  newFileName = fileName[0:fileName.find(".")]    

  testGene = testGenerator(newFileName,file_path)

  model =  Unet(BACKBONE, encoder_weights='imagenet')
  model.load_weights("weight_1.hdf5")

  results = model.predict_generator(testGene,1,verbose=1)

  saveResult(newFileName,"./data/result7",results)

1/1 [==============================] - 4s 4s/step


1/1 [==============================] - 2s 2s/step


1/1 [==============================] - 1s 1s/step


1/1 [==============================] - 1s 1s/step


1/1 [==============================] - 1s 1s/step


...

1/1 [==============================] - 1s 1s/step


1/1 [==============================] - 1s 1s/step


1/1 [==============================] - 1s 1s/step


1/1 [==============================] - 1s 1s/step


1/1 [==============================] - 1s 1s/step


# **ROI detect**

In [ ]:
import cv2
import numpy

image_name = "T0001"
image_path = "./data/test"
path = os.path.join(image_path,"%s.png"%image_name)
image = cv2.imread(path)
image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
print(image.shape)
print(image.size)

(512, 512, 3)
786432


In [ ]:
mask_name = "T0001_predict"
mask_path = "./data/test"
path = os.path.join(mask_path,"%s.png"%mask_name)
mask = cv2.imread(path)
mask = cv2.cvtColor(mask,cv2.IMREAD_GRAYSCALE)
print(mask.shape)
print(mask.size)

(512, 512, 4)
1048576


**Single image for test**

*   Input: full pathes of image and mask 
*   Output: ROI and image with highlight



In [ ]:
import numpy as np
import cv2


original_mask = cv2.imread("./data/test/T0001_predict.png")
original_image = cv2.imread("./data/test/T0001.png")
mask = original_mask.copy()
image = original_image.copy()
gray = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
blurred = cv2.GaussianBlur(gray, (3, 3), 0)
canny = cv2.Canny(blurred, 120, 255, 1)

# Find contours in the image
cnts = cv2.findContours(canny.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
cnts = cnts[0] if len(cnts) == 2 else cnts[1]

# Obtain area for each contour
contour_sizes = [(cv2.contourArea(contour), contour) for contour in cnts]

# Find maximum contour and crop for ROI section
if len(contour_sizes) > 0:
    largest_contour = max(contour_sizes, key=lambda x: x[0])[1]
    x,y,w,h = cv2.boundingRect(largest_contour)
    cv2.rectangle(image, (x, y), (x + w, y + h), (36,255,12), 2)
    ROI = original_image[y:y+h, x:x+w]
    ROI = cv2.resize(ROI,(512,512), interpolation=cv2.INTER_CUBIC) 
    #cv2.imshow("ROI", ROI) 
    cv2.imwrite('./data/test/ROI.png', ROI, [cv2.IMWRITE_PNG_COMPRESSION, 0])

#cv2.imshow("canny", canny) 
#cv2.imwrite('./data/test/canny.png', canny, [cv2.IMWRITE_PNG_COMPRESSION, 0])
#cv2.imshow("detected", mask) 
cv2.imwrite('./data/test/detected.png', image, [cv2.IMWRITE_PNG_COMPRESSION, 0])

cv2.imwrite('./data/test/blurred.png',blurred, [cv2.IMWRITE_PNG_COMPRESSION, 0])

#cv2.waitKey(0)

True

**ROI Segmentation for multi images**

*   Input: pathes of images and masks folders
*   Output: ROIs and images with highlight

In [ ]:
import numpy as np
import cv2
def ROIsegement(mask_path,image_path,mask_name):
  image_name = mask_name[0:mask_name.find("_")]+".png"
  images_dir = os.path.join(image_path, image_name)
  masks_dir = os.path.join(mask_path, mask_name)
  original_mask = cv2.imread(masks_dir)
  original_image = cv2.imread(images_dir)
  mask = original_mask.copy()
  image = original_image.copy()
  gray = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
  blurred = cv2.GaussianBlur(gray, (3, 3), 0)
  canny = cv2.Canny(blurred, 120, 255, 1)

  # Find contours in the image
  cnts = cv2.findContours(canny.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
  cnts = cnts[0] if len(cnts) == 2 else cnts[1]

  # Obtain area for each contour
  contour_sizes = [(cv2.contourArea(contour), contour) for contour in cnts]

  # Find maximum contour and crop for ROI section
  if len(contour_sizes) > 0:
      largest_contour = max(contour_sizes, key=lambda x: x[0])[1]
      x,y,w,h = cv2.boundingRect(largest_contour)
      cv2.rectangle(image, (x, y), (x + w, y + h), (36,255,12), 2)
      ROI = original_image[y:y+h, x:x+w]
      ROI = cv2.resize(ROI,(512,512), interpolation=cv2.INTER_CUBIC) 
      ROI_name = mask_name[0:mask_name.find("_")]+"_ROI.png"
      ROI_dir = os.path.join("./data/ROI", ROI_name) 
      cv2.imwrite(ROI_dir, ROI, [cv2.IMWRITE_PNG_COMPRESSION, 0])

  #cv2.imshow("canny", canny) 
  #cv2.imwrite('./data/test/canny.png', canny, [cv2.IMWRITE_PNG_COMPRESSION, 0])
  #cv2.imshow("detected", mask) 
  detect_name = mask_name[0:mask_name.find("_")]+"_detect.png"
  detect_dir = os.path.join("./data/detect", detect_name)
  cv2.imwrite(detect_dir, image, [cv2.IMWRITE_PNG_COMPRESSION, 0])
  #cv2.waitKey(0)

In [ ]:
mask_path ="./data/result_test" 
image_path = "./data/val/image"
for fileName in os.listdir(mask_path):
  ROIsegement(mask_path,image_path,fileName)